In [44]:
import csv

import numpy as np
import rasterio
from sklearn.model_selection import train_test_split
from tensorflow import keras

from functions.image import get_filtered_satellite_float32, get_satellite_raw_uint8

In [5]:
dataset = rasterio.open("./data/crs.tiff")
img_data = dataset.read(1)
block_size = 30


def get_image(coord):
    return get_filtered_satellite_float32(coord, block_size, img_data, dataset)

In [6]:
with open('training/data_bin.csv', 'r') as f:
    reader = csv.DictReader(f)
    labeled_coords = [r for r in reader]
print("labeled dataset contains {} points".format(len(labeled_coords)))

labeled dataset contains 115 points


In [7]:
X = list(map(get_image, labeled_coords))
Y = [int(l['windmill']) for l in labeled_coords]

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

X_train = np.stack(X_train)
X_test = np.stack(X_test)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

In [9]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2021-12-09 11:25:41.958538: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 11:25:41.966931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/rudolf/Projects/cs-ml-windmill/venv/lib/python3.9/site-packages/cv2/../../lib64:
2021-12-09 11:25:41.966954: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-12-09 11:25:41.967392: I tensorflow/core/platform/cpu_

In [10]:
model.fit(X_train, Y_train, epochs=10)

2021-12-09 11:25:43.286552: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
3/3 [==============================] - 0s 14ms/step - loss: 0.6531 - accuracy: 0.6250
Epoch 2/10
3/3 [==============================] - 0s 10ms/step - loss: 0.5736 - accuracy: 0.6500
Epoch 3/10
3/3 [==============================] - 0s 10ms/step - loss: 0.5177 - accuracy: 0.6625
Epoch 4/10
3/3 [==============================] - 0s 10ms/step - loss: 0.4843 - accuracy: 0.6750
Epoch 5/10
3/3 [==============================] - 0s 9ms/step - loss: 0.4383 - accuracy: 0.7625
Epoch 6/10
3/3 [==============================] - 0s 8ms/step - loss: 0.4021 - accuracy: 0.8250

In [11]:
model.evaluate(X_test, Y_test, verbose=2)

2/2 - 0s - loss: 0.1525 - accuracy: 0.9714


[0.15250590443611145, 0.9714285731315613]

In [118]:
left_top_coord = (0.32722,52.29054)  # enter coordinate here
img_h, img_w = (750, 500)
left_top_x, left_top_y = dataset.index(left_top_coord[0], left_top_coord[1])
needles = []
step_size = 8

for xx in range(left_top_x + block_size // 2, left_top_x + img_w - block_size // 2, step_size):
    for yy in range(left_top_y + block_size // 2, left_top_y + img_h - block_size // 2, step_size):
        cc = dataset.transform * (yy, xx)
        needles.append({'lon': cc[0], 'lat': cc[1]})

needle_data = np.stack([get_image(coord) for coord in needles])
results = model.predict(needle_data)

In [119]:
from PIL import Image

j = 0

for i, result in enumerate(results):
    if result[1] < result[0]:
        continue
    coord = needles[i]
    a = get_satellite_raw_uint8(coord, block_size, img_data, dataset)
    im = Image.fromarray(a)
    im.save("./examples/%s-%s.png" % (str(coord["lon"]).replace(".", "_"), str(coord["lat"]).replace(".", "_")))
    if j > 200:
        break
    j += 1